# Criminalità Italia
Analisi concentrata su numero dei delitti in Italia

## Fonte Dati
   1. Istat
       - http://dati.istat.it/Index.aspx
       - http://jsonstatpy.readthedocs.io/en/latest/notebooks/istat_house_price_index.html
   2. ...


In [1]:
# Import
import os
import pandas as pd
# pip install jsonstat.py --> Per analizzare dati istat e altri https://json-stat.org/
# http://jsonstatpy.readthedocs.io/en/latest/index.html
import istat

In [2]:
# Imposto cache dir
cache_dir = os.path.abspath(os.path.join("..", "tmp", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  #italiano
print("cache_dir is '{}'".format(istat.cache_dir()))

cache_dir is '/home/acesarini/opendata/tmp/istat_cached'


In [3]:
# Lista Aree - http://dati.istat.it/Index.aspx
# istat.areas()

In [4]:
# AREA Giustizia e Sicurezza
istat_area_lab = istat.area('JUS')
# istat_area_lab

In [5]:
# DATASET delitti http://dati.istat.it/Index.aspx?DataSetCode=dccv_delittips
istat_dataset_taxdisoccu = istat_area_lab.dataset('DCCV_DELITTIPS')
istat_dataset_taxdisoccu

nr,name,nr. values,values (first 3 values)
0,Territorio,148,"1:'Italia', 3:'Nord', 4:'Nord-ovest' ..."
1,Tipo dato,3,"1:'numero di delitti denunciati dalle forze di polizia all'autorità giudiziaria', 2:'delitti di cui si è scoperto l'autore nel corso dell'anno di riferimento (valori percentuali sul totale dei delitti dell'anno di riferimento)', 3:'delitti denunciati dalle forze di polizia all'autorità giudiziaria (valori per 100.000 abitanti)' ..."
2,Tipo di delitto,56,"1:'strage', 2:'omicidi volontari consumati', 3:'omicidi volontari consumati a scopo di furto o rapina' ..."
3,Identità autore nota,2,"2:'si', 3:'totale'"
4,Periodo del commesso delitto,3,"1:'prima dell'anno di riferimento', 2:'durante l'anno di riferimento', 3:'prima o durante l'anno di riferimento' ..."
5,Anno,9,"2129:'2012', 2034:'2007', 2148:'2013' ..."


In [10]:
# Lista di tutte le Dimensioni con Valori -> Necessari per creare il dataset via le spec
# istat_dataset_taxdisoccu.dimensions()

In [11]:
# Es. Numero crimini registrati dalla polizia(Data Type) per Anno 2014 (Year) su tutto il Territorio Italia (Territory), Omicidi Intenzionali (Type of crime)
spec = {
    "Tipo dato":1,                      # 1 -> number of crimes reported by the police forces to the judicial authority
    "Anno":2167,                        # 2167 -> 2014
    #"Territory":,                      # 0 -> ALL
    #"Type of crime":,                  # 0 -> ALL   
    "Identità autore nota":3,           # 3 -> Total
    "Periodo del commesso delitto":2    # 2 -> during the reference year
}

collection = istat_dataset_taxdisoccu.getvalues(spec)

In [164]:
# Lista dei dataset presenti nella collection
# collection

In [12]:
# Jsonstat - Dataset
ds = collection.dataset(0)
ds

pos,id,label,size,role
0,IDITTER107,Territorio,133,
1,IDREATI_PS,Tipo di delitto,56,


In [13]:
# Trasformo dataset in Dataframe
df = ds.to_data_frame('Territorio')

In [14]:
# Numero di delitti sul Territorio italiano, suddivisi per categoria (tot. 56 categorie)
df.head(56)

,Tipo di delitto,Value
Territorio,,
Italia,strage,20.0
Italia,omicidi volontari consumati,475.0
Italia,omicidi volontari consumati a scopo di furto o...,27.0
Italia,omicidi volontari consumati di tipo mafioso,45.0
Italia,omicidi volontari consumati a scopo terroristico,NaN
Italia,tentati omicidi,1250.0
Italia,infanticidi,3.0
Italia,omicidi preterintenzionali,34.0
Italia,omicidi colposi,1633.0
